In [1]:
import json 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import cv2
import os

In [2]:
path_to_ballet_2 ='C:\\Users\\user\\Desktop\\project\\1453169-20200914T174236Z-001\\TB_S'
path_9='C:\\Users\\user\\Desktop\\project\\1453169-20200914T174236Z-001\\SYN_B'

In [3]:
path_to_contemporary_2='C:\\Users\\user\\Desktop\\project\\1453169-20200914T174236Z-001\\TB_S_FB'

In [4]:
path_to_ballet = 'C:\\Users\\user\\Desktop\\project\\output\\ballet'
path_to_contemporary='C:\\Users\\user\\Desktop\\project\\output\\contemporary'
path_to_contemporary2='C:\\Users\\user\\Desktop\\project\\output\\contemporary21'
path_to_contemporary3='C:\\Users\\user\\Desktop\\project\\output\\contemporary22'

In [5]:
def read_keypoints(data):
    people_column=data['people']
    for d in people_column:
        keypoints=(d["pose_keypoints_2d"])
    return keypoints  


def remove_confidence_interval(data):

    j=2
    keypoints=read_keypoints(data)
    keypoints_woconfidence=keypoints.copy()
    while j<=len(keypoints_woconfidence):
        keypoints_woconfidence.pop(j)
        j+=2
    return keypoints_woconfidence

def create_y_coordicate(data):
    df=remove_confidence_interval(data)
    df=np.array(df)
    mask = np.ones(df.size, dtype=bool)
    mask[1::2] = 0
    points = df[mask]
    y=points.tolist()
    return y

def create_x_coordicate(data):
    df=remove_confidence_interval(data)
    df=np.array(df)
    mask = np.zeros(df.size, dtype=bool)
    mask[1::2] = 1
    points = df[mask]
    x=points.tolist()
    return x

def bodyparts(data):        
    
    # //     {0,  "Nose"}, 
    # //     {1,  "Neck"},
    # //     {2,  "RShoulder"},
    # //     {3,  "RElbow"},
    # //     {4,  "RWrist"},
    # //     {5,  "LShoulder"},
    # //     {6,  "LElbow"},
    # //     {7,  "LWrist"},
    # //     {8,  "MidHip"},
    # //     {9,  "RHip"},
    # //     {10, "RKnee"},
    # //     {11, "RAnkle"},
    # //     {12, "LHip"},
    # //     {13, "LKnee"},
    # //     {14, "LAnkle"},
    # //     {15, "REye"},
    # //     {16, "LEye"},
    # //     {17, "REar"},
    # //     {18, "LEar"},
    # //     {19, "LBigToe"},
    # //     {20, "LSmallToe"},
    # //     {21, "LHeel"},
    # //     {22, "RBigToe"},
    # //     {23, "RSmallToe"},
    # //     {24, "RHeel"},
    # //     {25, "Background"}    
    
    x=create_x_coordicate(data)
    y=create_y_coordicate(data)
    

    nose_to_neck=(x[0]-x[1]),(y[0]-y[1])
    
    right_eye_to_nose=(x[0]-x[15]),(y[0]-y[15])
    
    right_ear_to_eye=(x[15]-x[17]),(y[15]-y[17])
    left_eye_to_nose=(x[0]-x[16]),(y[0]-y[16])
    left_ear_to_eye=(x[16]-x[18]),(y[16]-y[18])
    
    torso= (x[1]-x[8]),(y[1]-y[8])
    
    
    right_shoulder=(x[1]-x[2]),(y[1]-y[2])
    left_shoulder=(x[1]-x[5]),(y[1]-y[5])
    
    right_upper_arm=(x[2]-x[3]),(y[2]-y[3])
    right_lower_arm=(x[3]-x[4]),(y[3]-y[4])
    left_upper_arm=(x[5]-x[6]),(y[5]-y[6])
    left_lower_arm=(x[6]-x[7]),(y[6]-y[7])
    
    right_hip=(x[8]-x[9]),(y[8]-y[9])
    right_upper_leg=(x[9]-x[10]),(y[9]-y[10])
    right_lower_leg=(x[10]-x[11]),(y[10]-y[11])
    left_hip=(x[8]-x[12]),(y[8]-y[12])
    left_upper_leg=(x[12]-x[13]),(y[12]-y[13])
    left_lower_leg=(x[13]-x[14]),(y[13]-y[14])
    
    left_foot=(x[14]-x[19]),(y[14]-y[19])
    left_toes=(x[19]-x[20]),(y[19]-y[20])
    left_ankle_to_heel=(x[14]-x[21]),(y[14]-y[21]) 
    right_foot=(x[11]-x[22]),(y[11]-y[22])
    right_toes=(x[22]-x[23]),(y[22]-y[23])
    right_ankle_to_heel=(x[11]-x[24]),(y[11]-y[24])
    
    body_vector=np.array([nose_to_neck, left_eye_to_nose ,left_ear_to_eye, right_eye_to_nose,right_ear_to_eye, torso,left_shoulder, right_shoulder,right_upper_arm ,right_lower_arm ,left_upper_arm ,left_lower_arm,left_hip ,left_upper_leg ,left_lower_leg ,right_hip ,right_upper_leg,right_lower_leg,right_foot ,right_toes ,right_ankle_to_heel ,left_foot ,left_toes ,left_ankle_to_heel])
    return body_vector


def similarity_bodyparts(data1,data2):
    #computes the similarity for every stick vector in the figures in one frame. 
    #I compute all the stick vectors per figure and then I do a pairwise comparison 
    #(also with cosine measure). 
    #I take the average of all similarities to come up with an overall similarity.
    bodyvector1=bodyparts(data1)
    bodyvector2=bodyparts(data2)
    cos_vector=np.array([])
    for i in range(len(bodyvector1)):
        a=cosine_similarity(bodyvector1[i].reshape(1,-1),bodyvector2[i].reshape(1,-1))
        #delete arrays with zeros
        if a != (np.array([0])):
            cos_vector=np.append(cos_vector,a )
        
    return round(np.mean(cos_vector),3) 

In [6]:
def create_df(path):
    newDF = pd.DataFrame(index=range(50))
    i=0
    
    
    for data in os.listdir(path):
        data=open(os.path.join(path,data),'r')
        data=json.load(data)
    
        def read_keypoints(data):
            people_column=data['people']
            for d in people_column:
                keypoints=(d["pose_keypoints_2d"])
            return keypoints  
    
    
        def remove_confidence_interval(data):
        
            j=2
            keypoints=read_keypoints(data)
            keypoints_woconfidence=keypoints.copy()
            while j<=len(keypoints_woconfidence):
                keypoints_woconfidence.pop(j)
                j+=2
            return keypoints_woconfidence
        
        
        bodyvector=remove_confidence_interval(data)
  
        new_bodyvector=pd.DataFrame(bodyvector)
        #print(new_bodyvector)
        
        
        #newDF['Frame: ',i]=new_bodyvector
        newDF[i]=new_bodyvector
        i+=1
        #print(bodyvector1)
        
    return newDF

In [7]:
create_df(path_9)

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,318.247,318.246,318.245,318.248,318.251,318.246,318.252,318.264,318.272,318.282,...,320.277,321.221,320.272,320.273,320.273,320.270,320.268,320.263,320.264,320.264
1,121.251,121.250,121.260,121.254,121.267,121.269,121.268,121.265,121.245,120.277,...,121.233,121.247,121.234,121.241,121.229,121.231,120.276,120.282,120.270,120.266
2,319.248,319.250,319.243,319.249,319.262,319.246,319.250,319.249,319.257,319.254,...,321.243,321.245,321.246,321.233,321.234,321.236,321.224,321.234,321.228,321.232
3,170.264,170.266,170.259,170.257,170.254,170.245,170.242,170.243,170.234,170.217,...,169.277,170.223,170.226,170.222,169.275,169.276,170.223,170.219,169.281,170.223
4,284.225,284.227,283.279,284.228,284.234,283.287,284.227,284.226,284.236,284.225,...,284.273,284.274,284.272,284.255,284.262,284.261,284.256,284.264,284.261,284.260
5,170.256,170.259,170.247,170.238,170.235,170.234,170.234,170.229,170.224,170.220,...,170.220,170.229,170.235,170.234,170.228,170.227,170.241,170.242,170.242,170.248
6,284.295,284.296,284.280,284.293,285.239,285.230,285.227,284.295,285.225,284.290,...,285.266,285.256,285.261,285.266,285.267,285.276,285.265,285.264,285.266,285.270
7,231.256,231.255,230.273,230.278,229.289,230.231,230.239,229.265,229.281,229.241,...,228.271,228.258,229.274,229.273,229.261,229.260,229.255,229.261,229.254,229.272
8,286.265,286.262,285.292,286.268,285.281,286.236,285.261,285.275,285.282,285.281,...,285.238,285.236,284.269,284.278,284.289,284.272,284.278,284.266,284.259,284.255
9,261.250,261.260,261.253,262.269,261.249,261.259,261.240,260.272,260.256,260.240,...,264.260,265.270,266.267,268.262,268.244,268.244,269.252,267.260,267.253,269.248


In [8]:
#X1=create_df(path_to_ballet)
#X2=create_df(path_to_contemporary)

In [9]:
X1=create_df(path_9)
X2=create_df(path_to_contemporary_2)

In [10]:
x1_coordinates=X1[::2]
x2_coordinates=X2[::2]

In [11]:
x2_coordinates

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,217.287,216.328,216.316,216.316,215.343,215.319,215.314,215.337,215.331,215.331,...,253.353,254.305,254.307,254.300,254.310,254.317,254.313,254.320,254.351,254.341
4,238.308,237.343,237.324,237.308,236.342,236.326,236.316,236.321,236.325,236.314,...,274.288,274.309,274.302,274.302,274.324,274.336,274.341,274.337,275.280,275.288
6,257.286,256.329,256.314,256.318,255.323,255.305,254.333,253.340,252.353,252.307,...,293.297,292.330,292.338,293.296,293.311,293.348,293.335,293.315,293.306,294.311
8,242.307,242.306,242.311,242.294,240.332,239.306,238.346,238.300,238.320,236.314,...,307.314,307.342,308.329,308.338,308.328,308.328,309.325,309.301,308.350,310.338
10,193.317,193.297,193.292,193.308,193.293,192.325,192.334,193.336,193.328,193.339,...,234.317,234.319,234.320,234.329,234.339,234.332,234.320,234.309,234.338,234.317
12,149.326,148.352,149.290,148.313,148.294,148.295,147.299,147.313,147.329,147.334,...,209.314,209.323,208.348,209.300,209.325,209.307,210.288,210.311,210.284,210.290
14,115.308,115.300,114.298,114.293,114.286,114.284,113.304,112.345,111.337,110.346,...,193.332,194.285,193.309,193.309,193.332,193.322,194.304,194.309,194.301,194.290
16,218.303,218.304,217.315,217.315,216.328,216.294,215.332,215.341,214.335,214.307,...,254.327,254.348,254.336,254.340,255.296,255.294,255.307,255.313,255.327,255.336
18,231.324,231.322,230.347,230.339,230.299,229.339,229.307,229.317,228.337,228.324,...,267.332,267.345,267.329,267.335,267.341,267.340,267.335,267.329,267.338,267.344


In [12]:
x1_coordinates

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,318.247,318.246,318.245,318.248,318.251,318.246,318.252,318.264,318.272,318.282,...,320.277,321.221,320.272,320.273,320.273,320.270,320.268,320.263,320.264,320.264
2,319.248,319.250,319.243,319.249,319.262,319.246,319.250,319.249,319.257,319.254,...,321.243,321.245,321.246,321.233,321.234,321.236,321.224,321.234,321.228,321.232
4,284.225,284.227,283.279,284.228,284.234,283.287,284.227,284.226,284.236,284.225,...,284.273,284.274,284.272,284.255,284.262,284.261,284.256,284.264,284.261,284.260
6,284.295,284.296,284.280,284.293,285.239,285.230,285.227,284.295,285.225,284.290,...,285.266,285.256,285.261,285.266,285.267,285.276,285.265,285.264,285.266,285.270
8,286.265,286.262,285.292,286.268,285.281,286.236,285.261,285.275,285.282,285.281,...,285.238,285.236,284.269,284.278,284.289,284.272,284.278,284.266,284.259,284.255
10,355.246,355.244,355.241,355.240,355.235,355.224,355.224,355.231,355.238,355.246,...,357.252,357.252,357.253,357.250,357.256,357.261,357.247,357.253,357.247,357.255
12,356.270,356.269,356.275,356.270,356.252,356.249,356.248,356.266,356.286,356.295,...,357.252,357.242,357.245,357.242,357.249,357.248,357.242,357.242,357.245,357.249
14,361.275,361.276,362.241,361.240,361.268,360.247,359.277,359.262,359.270,361.266,...,367.248,367.241,367.235,366.289,367.237,367.234,367.233,367.236,367.237,367.238
16,320.233,320.239,319.250,319.241,318.271,318.265,319.238,319.261,319.230,318.231,...,319.271,319.280,319.279,319.265,319.274,319.268,319.254,319.256,319.269,320.227
18,300.234,300.239,299.234,299.237,298.279,298.276,299.231,299.238,299.233,297.269,...,299.276,299.276,299.278,299.270,299.272,299.268,299.261,299.268,299.266,299.274


In [13]:
y1_coordinates=X1[1::2]
y2_coordinates=X2[1::2]

In [15]:
from tslearn.metrics import dtw, dtw_path
dtw(X1,X2)

5372.339172613242

In [16]:
path=dtw_path(X1,X2)[0]

In [17]:
L=path

In [14]:
L=[(0, 0),
 (1, 1),
 (2, 1),
 (3, 2),
 (4, 3),
 (5, 4),
 (6, 5),
 (7, 6),
 (8, 7),
 (9, 8),
 (10, 9),
 (11, 10),
 (12, 11),
 (13, 12),
 (14, 13),
 (15, 14),
 (16, 14),
 (17, 15),
 (18, 16),
 (19, 17),
 (20, 18),
 (21, 19),
 (22, 20),
 (23, 20),
 (24, 20),
 (25, 20),
 (26, 20),
 (27, 21),
 (28, 22),
 (29, 23),
 (30, 24),
 (30, 25),
 (31, 26),
 (32, 26),
 (33, 27),
 (34, 28),
 (35, 29),
 (36, 30),
 (37, 31),
 (38, 32),
 (39, 33),
 (40, 33),
 (41, 33),
 (42, 34),
 (43, 35),
 (44, 36),
 (45, 37),
 (46, 37),
 (47, 38),
 (48, 39),
 (49, 40),
 (50, 41),
 (51, 42),
 (52, 43),
 (53, 44),
 (54, 45),
 (55, 46),
 (56, 46),
 (57, 47),
 (58, 47),
 (59, 47),
 (60, 47),
 (61, 48),
 (62, 48),
 (63, 48),
 (64, 49),
 (65, 50),
 (66, 51),
 (67, 51),
 (68, 51),
 (69, 52),
 (70, 53),
 (71, 54),
 (72, 55),
 (73, 56),
 (74, 57),
 (75, 58),
 (76, 59),
 (77, 60),
 (78, 61),
 (79, 62),
 (80, 63),
 (81, 64),
 (82, 65),
 (83, 66),
 (84, 67),
 (85, 68),
 (86, 69),
 (87, 70),
 (88, 71),
 (89, 72),
 (90, 73),
 (91, 74),
 (92, 75),
 (93, 76),
 (94, 77),
 (95, 78),
 (96, 79),
 (97, 80),
 (97, 81),
 (97, 82),
 (97, 83),
 (97, 84),
 (97, 85),
 (97, 86),
 (97, 87),
 (97, 88),
 (97, 89),
 (97, 90),
 (97, 91),
 (97, 92),
 (97, 93),
 (97, 94),
 (97, 95),
 (97, 96),
 (97, 97),
 (97, 98),
 (97, 99),
 (98, 100)]

# Similarity for alligned frames

συγκρινεις Χ1,Χ2 που εχουν και χ και y αλλα δε σε νοιαζει γιατι το κανει pairwise ouτως ή αλλως.
το μηκος δεν ειναι το ιδιο...πολλα frames αντιστοιχουν σε ενα. αρα ειναι πανω απο 122 οι αντιστοιχισεις

In [62]:
len(X2)

50

In [19]:
similar_num=0
cos_vector=np.array([])
for i,j in L:
    a=cosine_similarity(X1[i].values.reshape(1,-1),X2[j].values.reshape(1,-1))
    #delete arrays with zeros
    if a != (np.array([0])):
        cos_vector=np.append(cos_vector,a )
        
print(len(cos_vector))
similar_num=round(np.mean(cos_vector),3) 

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 39 while Y.shape[1] == 101

In [20]:
similar_num

0

In [64]:
cos_vector

array([0.95018651, 0.95038459, 0.95102381, 0.95144317, 0.95189914,
       0.95246311, 0.93925203, 0.94132997, 0.94524226, 0.94585528,
       0.95607098, 0.95654477, 0.95125209, 0.95855404, 0.9582806 ,
       0.93630951, 0.16001209, 0.97131848, 0.99498804, 0.99551869,
       0.97225255, 0.95869709, 0.93862097, 0.93834303, 0.97293013,
       0.99688682, 0.99699676, 0.99767647, 0.97563845, 0.99795001,
       0.9632737 , 0.96337769, 0.99818138, 0.92738162, 0.99720206,
       0.96693974, 0.99657439, 0.99557954, 0.99574029, 0.99557205,
       0.98349153, 0.99619727, 0.99620222, 0.94871927, 0.20135696,
       0.92129111, 0.92423723, 0.91944029, 0.92706479, 0.90169633,
       0.97142297, 0.95643406, 0.98413595, 0.98404554, 0.98430693,
       0.99854138, 0.98482274, 0.98509336, 0.9991084 , 0.99915028,
       0.99924378, 0.99933283, 0.99919919, 0.99923655, 0.99889602,
       0.98606057, 0.98569257, 0.9855928 , 0.98543407, 0.98543977,
       0.97180094, 0.97134393, 0.9707071 , 0.97041687, 0.98288

In [65]:
np.max(cos_vector)

0.9993328340928246

In [66]:
np.argmax(cos_vector)

61

61th frame of video1.

Most important frames that contribute to similarity??

In [67]:
cos_vector.argsort()[-10:][::-1]

array([61, 60, 63, 62, 59, 58, 64, 55, 32, 29], dtype=int64)

How to determine the most important angle in terms of similarity?


In [68]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
pca.fit(X1)
PCA(n_components=2)
print(pca.explained_variance_ratio_)

print(pca.singular_values_)


[0.75374833 0.10020958 0.05669691 0.02209361 0.0170967  0.0154332
 0.0100103  0.00724448 0.00494226 0.00337795]
[4976.44101059 1814.51394072 1364.85135892  851.9989144   749.48313656
  712.08800858  573.49446132  487.87576067  402.96592201  333.14423233]
